# Homework 6
Molly McFaul

## Question 1

### Functions Defined

In [98]:
import numpy as np
import math
import time
from numpy.linalg import norm 
from numpy.linalg import inv
from numpy.linalg import det
import matplotlib.pyplot as plt
from numpy.linalg import norm

def evalF(x): 

    F = np.zeros(2)
    
    F[0] = x[0]**2 + x[1]**2 - 4
    F[1] = np.exp(x[0]) + x[1] - 1
    return F

def evalJ(x): 

    J = np.array([[2*x[0],2*x[1]], 
        [np.exp(x[0]), 1]])
    return J

def Newton(x0,tol,Nmax):
    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''
    if det(evalJ(x0)) == 0:
        xstar = x0
        ier = 1
        its = 'singular matrix'
        return [xstar, ier, its]
    
    
    for its in range(Nmax):
       J = evalJ(x0)
       Jinv = inv(J)
       F = evalF(x0)
       
       x1 = x0 - Jinv.dot(F)
       
       if (norm(x1-x0) < tol):
           xstar = x1
           ier =0
           return[xstar, ier, its]
           
       x0 = x1
    
    xstar = x1
    ier = 1
    return[xstar,ier,its]

def LazyNewton(x0,tol,Nmax):

    ''' Lazy Newton = use only the inverse of the Jacobian for initial guess'''
    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''
    if det(evalJ(x0)) == 0:
        xstar = x0
        ier = 1
        its = 'singular matrix'
        return [xstar, ier, its]
    J = evalJ(x0)
    Jinv = inv(J)
    for its in range(Nmax):

       F = evalF(x0)
       x1 = x0 - Jinv.dot(F)
       
       if (norm(x1-x0) < tol):
           xstar = x1
           ier =0
           return[xstar, ier,its]
           
       x0 = x1
    
    xstar = x1
    ier = 1
    return[xstar,ier,its] 


def SlackerNewton(x0, tol, Nmax):
    if det(evalJ(x0)) == 0:
        xstar = x0
        ier = 1
        its = 'singular matrix'
        return [xstar, ier, its]
    
    J = evalJ(x0)
    Jinv = inv(J)
    if det(evalJ(x0)) == 0:
        xstar = x0
        ier = 1
        its = 'singular matrix'
        return [xstar, ier, its]
    for its in range(Nmax):
       F = evalF(x0)
       
       x1 = x0 - Jinv.dot(F)
       
       if (norm(x1-x0) < tol):
           xstar = x1
           ier =0
           return[xstar, ier, its]
       
       if its % 3 == 0: #On every 3rd iteration, recompute Jacobian
        J = evalJ(x1)
        Jinv = inv(J)
        
       x0 = x1
    
    xstar = x1
    ier = 1
    return[xstar,ier,its]

def Broyden(x0,tol,Nmax):
    '''tol = desired accuracy
    Nmax = max number of iterations'''

    '''Sherman-Morrison 
   (A+xy^T)^{-1} = A^{-1}-1/p*(A^{-1}xy^TA^{-1})
    where p = 1+y^TA^{-1}Ax'''

    '''In Newton
    x_k+1 = xk -(G(x_k))^{-1}*F(x_k)'''


    '''In Broyden 
    x = [F(xk)-F(xk-1)-\hat{G}_k-1(xk-xk-1)
    y = x_k-x_k-1/||x_k-x_k-1||^2'''

    ''' implemented as in equation (10.16) on page 650 of text'''
    
    '''initialize with 1 newton step'''
    
    A0 = evalJ(x0)
    
    if det(evalJ(x0)) == 0:
        xstar = x0
        ier = 1
        its = 'singular matrix'
        return [xstar, ier, its]
    
    v = evalF(x0)
    A = np.linalg.inv(A0)

    s = -A.dot(v)
    xk = x0+s
    for  its in range(Nmax):
       '''(save v from previous step)'''
       w = v
       ''' create new v'''
       v = evalF(xk)
       '''y_k = F(xk)-F(xk-1)'''
       y = v-w;                   
       '''-A_{k-1}^{-1}y_k'''
       z = -A.dot(y)
       ''' p = s_k^tA_{k-1}^{-1}y_k'''
       p = -np.dot(s,z)                 
       u = np.dot(s,A) 
       ''' A = A_k^{-1} via Morrison formula'''
       tmp = s+z
       tmp2 = np.outer(tmp,u)
       A = A+1./p*tmp2
       ''' -A_k^{-1}F(x_k)'''
       s = -A.dot(v)
       xk = xk+s
       if (norm(s)<tol):
          alpha = xk
          ier = 0
          return[alpha,ier,its]
    alpha = xk
    ier = 1
    return[alpha,ier,its]

### Driver functions for Newton with different initial guesses

In [99]:
def driver1N():
    x0 = np.array([1.,1.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(50):
      [xstar,ier,its] =  Newton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Newton(1,1): the error message reads:',ier) 
    print('Newton(1,1): took this many seconds:',elapsed/50)
    print('Netwon(1,1): number of iterations is:',its)
def driver2N():
    x0 = np.array([1.,-1.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(50):
      [xstar,ier,its] =  Newton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Newton(1,-1): the error message reads:',ier) 
    print('Newton(1,-1): took this many seconds:',elapsed/50)
    print('Netwon(1,-1): number of iterations is:',its)
def driver3N():
    x0 = np.array([0.,0.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(50):
      [xstar,ier,its] =  Newton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Newton(0,0): the error message reads:',ier) 
    print('Newton(0,0): took this many seconds:',elapsed/50)
    print('Netwon(0,0): number of iterations is:',its)

In [100]:
driver1N()
driver2N()
driver3N()

[-1.81626407  0.8373678 ]
Newton(1,1): the error message reads: 0
Newton(1,1): took this many seconds: 0.000712895393371582
Netwon(1,1): number of iterations is: 7
[ 1.00416874 -1.72963729]
Newton(1,-1): the error message reads: 0
Newton(1,-1): took this many seconds: 0.00016145706176757814
Netwon(1,-1): number of iterations is: 5
[0. 0.]
Newton(0,0): the error message reads: 1
Newton(0,0): took this many seconds: 0.0
Netwon(0,0): number of iterations is: singular matrix


### Driver functions for Lazy Newton with different initial guesses

In [101]:
def driver1LN():
    x0 = np.array([1.,1.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(20):
      [xstar,ier,its] =  LazyNewton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Lazy Newton(1,1): the error message reads:',ier)
    print('Lazy Newton(1,1): took this many seconds:',elapsed/20)
    print('Lazy Newton(1,1): number of iterations is:',its)
def driver2LN():
    x0 = np.array([1.,-1.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(20):
      [xstar,ier,its] =  LazyNewton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Lazy Newton(1,-1): the error message reads:',ier)
    print('Lazy Newton(1,-1): took this many seconds:',elapsed/20)
    print('Lazy Newton(1,-1): number of iterations is:',its)
def driver3LN():
    x0 = np.array([0.,0.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(20):
      [xstar,ier,its] =  LazyNewton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Lazy Newton(0,0): the error message reads:',ier)
    print('Lazy Newton(0,0): took this many seconds:',elapsed/20)
    print('Lazy Newton(0,0): number of iterations is:',its)

    

### Driver functions for Broyden with different initial guesses

In [102]:
def driver1B():
    
    x0 = np.array([1.,1.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(20):
      [xstar,ier,its] = Broyden(x0, tol,Nmax)     
    elapsed = time.time()-t
    print(xstar)
    print('Broyden(1,1): the error message reads:',ier)
    print('Broyden(1,1): took this many seconds:',elapsed/20)
    print('Broyden(1,1): number of iterations is:',its)
    
def driver2B():
    
    x0 = np.array([1.,-1.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(20):
      [xstar,ier,its] = Broyden(x0, tol,Nmax)     
    elapsed = time.time()-t
    print(xstar)
    print('Broyden(1,-1): the error message reads:',ier)
    print('Broyden(1,-1): took this many seconds:',elapsed/20)
    print('Broyden(1,-1): number of iterations is:',its)
    
def driver3B():
    
    x0 = np.array([0.,0.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(20):
      [xstar,ier,its] = Broyden(x0, tol,Nmax)     
    elapsed = time.time()-t
    print(xstar)
    print('Broyden(0,0): the error message reads:',ier)
    print('Broyden(0,0): took this many seconds:',elapsed/20)
    print('Broyden(0,0): number of iterations is:',its)
    

### Driver functions for Slacker Newton with different initial guesses

In [103]:
def driver1SN():
    x0 = np.array([1.,1.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(50):
      [xstar,ier,its] =  SlackerNewton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Slacker Newton(1,1): the error message reads:',ier) 
    print('Slacker Newton(1,1): took this many seconds:',elapsed/50)
    print('Slacker Netwon(1,1): number of iterations is:',its)   
def driver2SN():
    x0 = np.array([1.,-1.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(50):
      [xstar,ier,its] =  SlackerNewton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Slacker Newton(1,-1): the error message reads:',ier) 
    print('Slacker Newton(1,-1): took this many seconds:',elapsed/50)
    print('Slacker Netwon(1,-1): number of iterations is:',its)  
def driver3SN():
    x0 = np.array([0.,0.])
    Nmax = 100
    tol = 1e-10
    
    t = time.time()
    for j in range(50):
      [xstar,ier,its] =  SlackerNewton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Slacker Newton(0,0): the error message reads:',ier) 
    print('Slacker Newton(0,0): took this many seconds:',elapsed/50)
    print('Slacker Netwon(0,0): number of iterations is:',its)  

### Calling Newton

In [104]:
driver1N()
driver2N()
driver3N()

[-1.81626407  0.8373678 ]
Newton(1,1): the error message reads: 0
Newton(1,1): took this many seconds: 0.00016044139862060548
Netwon(1,1): number of iterations is: 7
[ 1.00416874 -1.72963729]
Newton(1,-1): the error message reads: 0
Newton(1,-1): took this many seconds: 0.00016115188598632812
Netwon(1,-1): number of iterations is: 5
[0. 0.]
Newton(0,0): the error message reads: 1
Newton(0,0): took this many seconds: 0.0
Netwon(0,0): number of iterations is: singular matrix


### Calling Lazy Newton

In [105]:
driver1LN()
driver2LN()
driver3LN()

[nan nan]
Lazy Newton(1,1): the error message reads: 1
Lazy Newton(1,1): took this many seconds: 0.0014200806617736816
Lazy Newton(1,1): number of iterations is: 99
[ 1.00416874 -1.72963729]
Lazy Newton(1,-1): the error message reads: 0
Lazy Newton(1,-1): took this many seconds: 0.0005782246589660645
Lazy Newton(1,-1): number of iterations is: 36
[0. 0.]
Lazy Newton(0,0): the error message reads: 1
Lazy Newton(0,0): took this many seconds: 0.0
Lazy Newton(0,0): number of iterations is: singular matrix


C:\Users\Molly\AppData\Local\Temp\ipykernel_19196\129197612.py:15: RuntimeWarning: overflow encountered in exp
  F[1] = np.exp(x[0]) + x[1] - 1


### Calling Broyden 

In [106]:
driver1B()
driver2B()
driver3B()

[-1.81626407  0.8373678 ]
Broyden(1,1): the error message reads: 0
Broyden(1,1): took this many seconds: 0.0005509018898010254
Broyden(1,1): number of iterations is: 12
[ 1.00416874 -1.72963729]
Broyden(1,-1): the error message reads: 0
Broyden(1,-1): took this many seconds: 0.00011757612228393554
Broyden(1,-1): number of iterations is: 6
[0. 0.]
Broyden(0,0): the error message reads: 1
Broyden(0,0): took this many seconds: 0.0
Broyden(0,0): number of iterations is: singular matrix


### Calling Slacker Newton

In [107]:
driver1SN()
driver2SN()
driver3SN()

[-1.81626407  0.8373678 ]
Slacker Newton(1,1): the error message reads: 0
Slacker Newton(1,1): took this many seconds: 0.00014298439025878907
Slacker Netwon(1,1): number of iterations is: 11
[ 1.00416874 -1.72963729]
Slacker Newton(1,-1): the error message reads: 0
Slacker Newton(1,-1): took this many seconds: 0.00035570621490478517
Slacker Netwon(1,-1): number of iterations is: 6
[0. 0.]
Slacker Newton(0,0): the error message reads: 1
Slacker Newton(0,0): took this many seconds: 0.0
Slacker Netwon(0,0): number of iterations is: singular matrix


For all of the methods, attempting to use the initial guess of $\overline{x}_0 = [0,0]$ results in the intial Jacobian calculated being singular, and thus renders the methods unable to continue. It can be noted that the point $(0,0)$ is a root of $g(x,y)$. For the initial guess of $\overline{x}_0=[1,1]$, the Lazy Newton method did not work. It outputs that it reached $N_{max}$ i.e., it performed 99 iterations. Python outputs an overflow error, the numerical value of the iteration being too large when evaluated the exponential in $g(x,y)$ -- the outputted roots are "nan" values. For Slacker Newton, where the Jacobian is recalculated every 3 iterations instead of every iteration, required slightly more iterations than the normal Newton's Method. Newton required 7 iterations for $\overline{x}_0 = [1,1]$ while Slacker Newton required 11 iterations. Broyden required 12 iterations for this initial guess, more than both Newton and Slacker Newton.  For $\overline{x}_0 = [1,-1]$, Newton required 5 iterations, Lazy Newton required 36 iterations, and Broyden required 6. Slacker Newton required 6, though only requiring a Jacobian calculation every three. 

# Question 2

### Defining Key Functions

In [108]:
def evalF(x): 

    F = np.zeros(3)
    
    F[0] = x[0] + math.cos(x[0]*x[1]*x[2]) - 1
    F[1] = (1 - x[0])**(1/4) + x[1] + 0.05*x[2]**2 - 0.15 * x[2] - 1
    F[2] = -1*(x[0]**2) - 0.1*x[1]**2 + 0.01*x[1] + x[2] - 1
    return F
def evalJ(x): 
    fx = 1 - x[1]*x[2]*math.sin(x[0]*x[1]*x[2])
    fy = - x[0]*x[2]*math.sin(x[0]*x[1]*x[2])
    fz = - x[1]*x[0]*math.sin(x[0]*x[1]*x[2])
    
    gx = -(1/4) * (1-x[0])**(-3/4)
    gy = 1
    gz = 0.1*x[2] - 0.15
    
    hx = -2*x[0]
    hy = -.2*x[1] + 0.01
    hz = 1
    
    J = np.array([[fx,fy,fz],[gx,gy,gz],[hx,hy,hz]])
    return J

def Newton(x0,tol,Nmax):
    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''
    if det(evalJ(x0)) == 0:
        xstar = x0
        ier = 1
        its = 'singular matrix'
        return [xstar, ier, its]
    
    
    for its in range(Nmax):
       J = evalJ(x0)
       Jinv = inv(J)
       F = evalF(x0)
       
       x1 = x0 - Jinv.dot(F)
       
       if (norm(x1-x0) < tol):
           xstar = x1
           ier =0
           return[xstar, ier, its]
           
       x0 = x1
    
    xstar = x1
    ier = 1
    return[xstar,ier,its]


def evalg(x):

    F = evalF(x)
    g = F[0]**2 + F[1]**2 + F[2]**2
    return g

def eval_gradg(x):
    F = evalF(x)
    J = evalJ(x)
    
    gradg = np.transpose(J).dot(F)
    return gradg


###############################
### steepest descent code

def SteepestDescent(x,tol,Nmax):
    
    for its2 in range(Nmax):
        g1 = evalg(x)
        z = eval_gradg(x)
        z0 = norm(z)

        if z0 == 0:
            print("zero gradient")
        z = z/z0
        alpha1 = 0
        alpha3 = 1
        dif_vec = x - alpha3*z
        g3 = evalg(dif_vec)

        while g3>=g1:
            alpha3 = alpha3/2
            dif_vec = x - alpha3*z
            g3 = evalg(dif_vec)
            
        if alpha3<tol:
            print("no likely improvement")
            ier = 0
            return [x,g1,ier,its2]
        
        alpha2 = alpha3/2
        dif_vec = x - alpha2*z
        g2 = evalg(dif_vec)

        h1 = (g2 - g1)/alpha2
        h2 = (g3-g2)/(alpha3-alpha2)
        h3 = (h2-h1)/alpha3

        alpha0 = 0.5*(alpha2 - h1/h3)
        dif_vec = x - alpha0*z
        g0 = evalg(dif_vec)

        if g0<=g3:
            alpha = alpha0
            gval = g0

        else:
            alpha = alpha3
            gval =g3

        x = x - alpha*z

        if abs(gval - g1)<tol:
            ier = 0
            return [x,gval,ier,its2]

    print('max iterations exceeded')    
    ier = 1        
    return [x,g1,ier,its2]      

### Driver Function for Newton with initial guess $[0,0,0]$

In [123]:
def driverN():
    x0 = np.array([-1,1,1])
    Nmax = 100
    tol = 1e-12
    
    t = time.time()
    for j in range(50):
      [xstar,ier,its] =  Newton(x0,tol,Nmax)
    elapsed = time.time()-t
    print(xstar)
    print('Newton: the error message reads:',ier) 
    print('Newton: took this many seconds:',elapsed/50)
    print('Newton: number of iterations is:',its)

### Driver Function for Steepest Descent with initial guess $[0,0,0]$

In [124]:
def driverSD():

    Nmax = 100
    x0= np.array([-1,1,1])
    tol = 1e-10
    
    [xstar,gval,ier,its] = SteepestDescent(x0,tol,Nmax)
    print(xstar)
    print("Steepest Descent: the error message reads", ier	)
    print('Steepest Descent: number of iterations is:', its)
    

In [125]:
def driverNSD():

    Nmax = 100
    x0= np.array([-1,1,1])
    tol = 5e-2
    
    for j in range(50):
      [xstar,ier,its] =  Newton(x0,tol,Nmax)
        
    tol = 1e-10
    x0 = xstar
    [xstar,gval,ier,its2] = SteepestDescent(x0,tol,Nmax)
    print(xstar)
    print("Newton + Steepest Descent: the error message reads", ier	)
    print('Newton + Steepest Descent: number of iterations is:', its + its2)

### Initial guess = $[-1,1,1]$, tol for final solution = $10^{-6}$, $N_{max} = 100$

### Calling Newton 

In [126]:
driverN()

[4.04637797e-17 1.00000000e-01 1.00000000e+00]
Newton: the error message reads: 0
Newton: took this many seconds: 0.0003883028030395508
Newton: number of iterations is: 6


The solution seems to be $[x,y,z] = [0,0.1,1]$. This technique converged in only 6 iterations. 

### Calling Steepest Descent 

In [127]:
driverSD()

[-6.56738397e-08  1.00000365e-01  1.00000003e+00]
Steepest Descent: the error message reads 0
Steepest Descent: number of iterations is: 8


The solution here seems to be less accurate than with Newton's method, though the tolerance was set to be the same. This technique converged in 8 iterations, two more than Newton's.

### Calling Newton + Steepest Descent

In [129]:
driverNSD()

[9.25244936e-11 9.99996879e-02 9.99999978e-01]
Newton + Steepest Descent: the error message reads 0
Newton + Steepest Descent: number of iterations is: 3


This solution seems to be slightly less accurate than Newton's method, though again the tolerance was set to be the same. However, the combination of Newton's and Steepest Descent led to convergence in the least amount of iterations, only 3, compared to SD alone with 8 iterations and Newton alone with 6 iterations. This makes sense, as Newton tends to converge the fastest, given that the initial guess is close to the exact root. If it isn't Newton does not perform very well as compared to other methods. With this combination, we get the quick convergence of Newton's method with a guaranteed close intial guess, as the Steepest Descent method can converge with a further initial guess than Newton's can. So, with an initial guess like $[-1,1,1]$, that is not wildly close to the solution, we would expect that this combination converges faster than both Newton's method and the steepest descent method alone. 